# VLM for KOMO

In [2]:
import time
import torch
import robotic as ry
import matplotlib.pyplot as plt

from utils import grasping_within_komo_definition

In [8]:
torch.cuda.empty_cache()

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device Name: {torch.cuda.get_device_name(device)}" if device.type == "cuda" else "Using cpu")

Using cpu


## Prepare your scene

In [3]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.06,.06,.005]) \
    .setColor([1,.5,0]) \
    .setContact(1)
C.view()

0

## Take a picture of the scene

In [4]:
bot = ry.BotOp(C, False)
bot.home(C)
bot.gripperMove(ry._left)

rgb, depth = bot.getImageAndDepth("cameraWrist")

image_path = "./scene_image.jpg"
plt.imsave(image_path, rgb)

## Define your model (VLM)

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype="auto",
    device_map=device
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

## Prompt the model (for KOMO)

In [ ]:
from prompting import prompt_qwen

task_description = "Put the blob in the bin."

komo_definition = prompt_qwen(model, processor, device, task_description, image_path)
print(komo_definition)

## Solve the komo problem

In [4]:
komo: ry.KOMO = None
exec(komo_definition)
ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
print(ret)

{ time: 0.000313, evals: 6, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00256402 }


## Visualize solution

In [5]:
qs = komo.getPath()

for q in qs:
    C.setJointState(q)
    C.view()
    time.sleep(1.)

## Execute the motion

In [ ]:
# Identify phases where we grasp
switch_indices = grasping_within_komo_definition(komo_definition)

# Execute on robot
prev_s = 0
for i, s in enumerate(switch_indices):
    
    # Moving
    bot.moveAutoTimed(qs[prev_s:s])
    while bot.getTimeToEnd() > 0:
        bot.sync(C)

    # Grasping
    if i % 2 == 0:
        bot.gripperClose(ry._left)
    else:
        bot.gripperMove(ry._left)
        
    prev_s = s